## 9. Comparing and Combining Models
Before comparing metrics or combining strategies, it's important to step back and assess what we're actually solving. Each strategy targets a different type of market behavior and expresses risk differently. Evaluating them in isolation provides clarity on their individual tradeoffs. But in practice, no strategy operates in a vacuum—returns, risk, and drawdowns must be assessed within the context of broader portfolio construction. The next sections move from individual performance to combined results to understand how these systems interact over time.

### 9.1. Individual Performance Metric Analysis
Both models—trend-following and mean reversion—demonstrate consistent performance from 2001 through mid-2025 when applied to futures markets. For context, the S&P 500 Total Return Index is included, though it differs structurally as it represents long-only equity exposure. Table 6 presents a detailed breakdown of performance metrics across all three, allowing for direct comparison across risk, return, and efficiency dimensions.

The mean reversion model delivered the strongest cumulative and annual returns, reaching 1178.65% and 11.42% respectively. This came with higher volatility at 14.43% and a larger max drawdown of -26.65%, though still far less than the S&P 500's -55.25%. This suggests the model maintains a relatively controlled risk profile even during major stress events.

The trend-following model, while lower in total return (633.09.80%) and annual return (8.82%), offered lower volatility (11.61%) and more contained drawdowns (-23.84%). This more defensive profile aligns with the strategy's nature: slower reaction, but more protection when stress rises.

In [ ]:
import pandas as pd
import numpy as np
import norgatedata
from scipy.stats import skew, kurtosis, linregress
from IPython.display import display, HTML

# === Parameters ===
mr_file = r'C:\Users\Juanan\Downloads\CQF\2.Mean_Reversion\1.Zipline_Trades\1.Full_Backtest\returns_2001-12-28_to_2025-08-12_MR.csv'
tf_file = r'C:\Users\Juanan\Downloads\CQF\1.Futures_Trend_Following\1.Zipline_Trades\1.Full_Backtest\returns_2001-12-28_to_2025-08-12_TF.csv'
benchmark_symbol = '$SPXTR'
start_date = '2001-12-31'
end_date = '2025-08-12'
risk_free_rate = 0.00  # daily assumed 0%

# === 1. Load Strategy Data ===
def load_returns(file_path):
    df = pd.read_csv(file_path)
    df.columns = ['Date', 'Returns']
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index('Date').sort_index()
    df = df.loc[start_date:end_date]
    return df['Returns']

mean_reversion = load_returns(mr_file)
trend_following = load_returns(tf_file)

# === 2. Load Benchmark ($SPXTR) ===
benchmark_df = norgatedata.price_timeseries(
    benchmark_symbol,
    start_date=start_date,
    end_date=end_date,
    format='pandas-dataframe',
    stock_price_adjustment_setting=norgatedata.StockPriceAdjustmentType.NONE,
    padding_setting=norgatedata.PaddingType.NONE,
    timezone='UTC'
)
benchmark_df = benchmark_df.dropna(subset=['Close'])
benchmark_returns = benchmark_df['Close'].pct_change().dropna()
benchmark_returns = benchmark_returns.loc[start_date:end_date]

# === 3. Merge Data (Align Dates) ===
df = pd.DataFrame({
    'Mean Reversion': mean_reversion,
    'Trend Following': trend_following,
    'Benchmark ($SPXTR)': benchmark_returns
}).dropna()

# === 4. Metrics Function ===
def compute_metrics(returns, benchmark=None):
    stats = {}
    daily_returns = returns
    annual_factor = 252
    cumulative = (1 + daily_returns).prod() - 1
    annual_return = (1 + cumulative) ** (annual_factor / len(daily_returns)) - 1
    volatility = daily_returns.std() * np.sqrt(annual_factor)
    sharpe = (daily_returns.mean() - risk_free_rate) / daily_returns.std() * np.sqrt(annual_factor)

    cumulative_returns = (1 + daily_returns).cumprod()
    rolling_max = cumulative_returns.cummax()
    drawdown = (cumulative_returns - rolling_max) / rolling_max
    max_dd = drawdown.min()
    calmar = annual_return / abs(max_dd) if max_dd != 0 else np.nan

    log_returns = np.log(cumulative_returns)
    x = np.arange(len(log_returns))
    slope, intercept, r_value, *_ = linregress(x, log_returns)
    stability = r_value ** 2

    threshold = 0.0
    excess = daily_returns - threshold
    downside = excess[excess < 0]
    downside_dev = np.sqrt(np.mean(downside ** 2))
    sortino = excess.mean() / downside_dev * np.sqrt(annual_factor) if downside_dev != 0 else np.nan

    omega = daily_returns[daily_returns > threshold].sum() / abs(daily_returns[daily_returns < threshold].sum())
    tail_ratio = np.percentile(daily_returns, 95) / abs(np.percentile(daily_returns, 5))
    var = np.percentile(daily_returns, 5)

    stats['Annual Return (%)'] = annual_return * 100
    stats['Cumulative Return (%)'] = cumulative * 100
    stats['Annual Volatility (%)'] = volatility * 100
    stats['Sharpe Ratio'] = sharpe
    stats['Calmar Ratio'] = calmar
    stats['Stability (R²)'] = stability
    stats['Max Drawdown (%)'] = max_dd * 100
    stats['Omega Ratio (0%)'] = omega
    stats['Sortino Ratio (0%)'] = sortino
    stats['Skew'] = skew(daily_returns)
    stats['Kurtosis'] = kurtosis(daily_returns)
    stats['Tail Ratio'] = tail_ratio
    stats['Daily VaR (95%)'] = var

    # Alpha & Beta (vs. benchmark)
    if benchmark is not None:
        aligned = pd.concat([daily_returns, benchmark], axis=1).dropna()
        slope, intercept, r, _, _ = linregress(aligned.iloc[:, 1], aligned.iloc[:, 0])
        stats['Beta'] = slope
        stats['Alpha (Ann.)'] = intercept * annual_factor
    else:
        stats['Beta'] = np.nan
        stats['Alpha (Ann.)'] = np.nan

    return stats

# === 5. Compute Metrics ===
mr_metrics = compute_metrics(df['Mean Reversion'], df['Benchmark ($SPXTR)'])
tf_metrics = compute_metrics(df['Trend Following'], df['Benchmark ($SPXTR)'])
bm_metrics = compute_metrics(df['Benchmark ($SPXTR)'])

# === 6. Display Table (Nicely Formatted) ===
results = pd.DataFrame({
    'Mean Reversion': mr_metrics,
    'Trend Following': tf_metrics,
    'Benchmark ($SPXTR)': bm_metrics
})

# Optional: Sort by rows in consistent order
preferred_order = [
    'Annual Return (%)', 'Cumulative Return (%)', 'Annual Volatility (%)', 'Sharpe Ratio', 'Calmar Ratio',
    'Stability (R²)', 'Max Drawdown (%)', 'Omega Ratio (0%)', 'Sortino Ratio (0%)',
    'Skew', 'Kurtosis', 'Tail Ratio', 'Daily VaR (95%)', 'Beta', 'Alpha (Ann.)'
]
results = results.loc[preferred_order]

# Pretty print
styled = results.round(2).style.set_caption("Strategy Performance Metrics (2001-12-31 to 2025-08-12)") \
    .format(precision=2) \
    .set_table_styles([{'selector': 'caption', 'props': [('font-size', '16px'), ('font-weight', 'bold')]}]) \
    .background_gradient(axis=1, cmap='Greys', subset=results.columns)

display(styled)


_Table 5 - Strategy Performance Metrics (2001-12-31 to 2025-08-12)_

Sharpe ratios are close between the two (0.82 for mean reversion vs. 0.79 for trend-following), both well within expectations for real-world systematic models. Sortino ratios are similarly balanced (0.75 and 0.72), indicating that neither model relies excessively on rare upside moves. Both strategies maintained strong path consistency, with stability scores (R²) of 0.98 and 0.94 respectively.

Daily return asymmetry is neutral across both models. Skew is mildly negative, and tail ratios hover near 1, suggesting gains and losses are balanced. Omega ratios remain above 1.1 in both models, which implies that positive returns are both more frequent and more significant than losses.

Beta values confirm both strategies are largely independent of equities (0.14 for mean reversion and 0.07 for trend-following), supporting their role in diversified portfolios. Alpha is positive in both models—0.10 and 0.08 respectively—indicating that excess return isn’t explained by exposure to the benchmark.

Combining both strategies improves portfolio construction. Their profiles are complementary: the trend-following model captures gains during sustained directional moves, while the mean reversion model tends to generate returns in sideways or volatile environments—particularly when trend systems are under pressure. Together, they reduce exposure to strategy-specific weaknesses and contribute to a more stable return profile.

This is illustrated in Figure 10, which shows the cumulative return curves of both models alongside the S&P 500 Total Return Index (SPXTR). The comparison provides a visual sense of how each model behaves across different market conditions.

In [ ]:
import pandas as pd
import numpy as np
import norgatedata
import matplotlib.pyplot as plt

# === Parameters ===
mr_file = r'C:\Users\Juanan\Downloads\CQF\2.Mean_Reversion\1.Zipline_Trades\1.Full_Backtest\returns_2001-12-28_to_2025-08-12_MR.csv'
tf_file = r'C:\Users\Juanan\Downloads\CQF\1.Futures_Trend_Following\1.Zipline_Trades\1.Full_Backtest\returns_2001-12-28_to_2025-08-12_TF.csv'
benchmark_symbol = '$SPXTR'
start_date = '2001-12-31'
end_date = '2025-08-12'

def load_returns(file_path, start_date, end_date):
    df = pd.read_csv(file_path)
    df = df.iloc[:, :2]
    df.columns = ['Date', 'Returns']
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index('Date').sort_index()
    df = df.loc[start_date:end_date]
    df['Returns'] = pd.to_numeric(df['Returns'], errors='coerce')
    return df['Returns'].dropna()

# 1) Load strategy daily returns
mr = load_returns(mr_file, start_date, end_date).rename('Mean Reversion')
tf = load_returns(tf_file, start_date, end_date).rename('Trend Following')

# 2) Load benchmark and compute returns
bm_prices = norgatedata.price_timeseries(
    benchmark_symbol,
    start_date=start_date,
    end_date=end_date,
    format='pandas-dataframe',
    stock_price_adjustment_setting=norgatedata.StockPriceAdjustmentType.NONE,
    padding_setting=norgatedata.PaddingType.NONE,
    timezone='UTC'
).dropna(subset=['Close'])
bm = bm_prices['Close'].pct_change().dropna().rename('Benchmark ($SPXTR)')

# 3) Align dates
rets = pd.concat([mr, tf, bm], axis=1).dropna()

# 4) Cumulative wealth
wealth = (1 + rets).cumprod()

# Mask any non-positive values (log scale requires > 0)
wealth = wealth.where(wealth > 0)

# 5) Plot (log scale)
plt.figure(figsize=(14, 6))
for col in wealth.columns:
    plt.plot(wealth.index, wealth[col], label=col)
plt.yscale('log')
plt.title('Cumulative Returns (Log Scale, Normalized to 1.0)')
plt.xlabel('Date')
plt.ylabel('')
plt.grid(True, which='both', axis='y')
plt.legend()
plt.tight_layout()
plt.show()



_Figure 10 - Models Equity Curve and SPXTR_

Further analysis should examine their combined behavior under capital and margin constraints to determine whether the joint implementation enhances the overall risk-adjusted return.

### 9.2. Combining the Models
Allocating capital equally between the mean reversion and trend-following systems and rebalancing the capital amount invested monthly, results in a portfolio that performs better than either model alone across most metrics. Monthly rebalancing keeps turnover manageable; yearly rebalancing is also reasonable and worth testing.

The combined model achieves an annual return of 10.48%, above the 8.82% from trend-following and closer to the 11.42% from mean reversion. More importantly, it does so with lower volatility than mean reversion (12.24% vs. 14.43%) and only a slight increase over the trend-following model 11.61%).

Sharpe and Sortino ratios improve compared to both standalone models, indicating better efficiency per unit of risk. The Calmar ratio also increases to 0.50, a clear improvement over the individual systems (0.43 for mean reversion and 0.38 for trend-following), reflecting a more favorable return-to-drawdown profile.

Max drawdown for the combined model is -20.81%, a marginal improvement over the trend-following system and substantially better than the -26.65% of mean reversion and the -55.25% of the benchmark.

Stability (R²) remains high at 0.98, indicating that the return profile is smoother and more predictable over time. The strategy also preserves desirable characteristics like a balanced omega ratio (1.17), improved beta (0.10), and a positive alpha (0.10), which confirms excess returns not explained by equity market exposure.

In [ ]:
import pandas as pd
import numpy as np
import norgatedata
from scipy.stats import skew, kurtosis, linregress
from IPython.display import display

# === Parameters ===
mr_file = r'C:\Users\Juanan\Downloads\CQF\2.Mean_Reversion\1.Zipline_Trades\1.Full_Backtest\returns_2001-12-28_to_2025-08-12_MR.csv'
tf_file = r'C:\Users\Juanan\Downloads\CQF\1.Futures_Trend_Following\1.Zipline_Trades\1.Full_Backtest\returns_2001-12-28_to_2025-08-12_TF.csv'
benchmark_symbol = '$SPXTR'
start_date = '2001-12-31'
end_date = '2025-08-12'
risk_free_rate = 0.00  # daily assumed 0%

# === 1. Load Strategy Data ===
def load_returns(file_path):
    df = pd.read_csv(file_path)
    df.columns = ['Date', 'Returns']
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index('Date').sort_index()
    df = df.loc[start_date:end_date]
    return df['Returns']

mean_reversion = load_returns(mr_file)
trend_following = load_returns(tf_file)

# === 2. Combine Strategies (50/50, Monthly Rebalance) ===
combined_df = pd.DataFrame({'MR': mean_reversion, 'TF': trend_following}).dropna()

class MonthRebalance:
    def __init__(self):
        self.last_month = None
    def rebalance(self, row, weights, date):
        if date.month != self.last_month:
            total = row.sum()
            row = np.array(weights) * total
            self.last_month = date.month
        return row

def calc_combined_returns(returns, weights=[0.5, 0.5]):
    rebalancer = MonthRebalance()
    cum = np.zeros_like(returns.values)
    cum[0] = np.array(weights)
    ret_vals = returns + 1
    for i in range(1, len(cum)):
        cum[i] = cum[i - 1] * ret_vals.iloc[i].values
        cum[i] = rebalancer.rebalance(cum[i], weights, returns.index[i])
    cum_df = pd.DataFrame(cum, index=returns.index, columns=returns.columns)
    rr = cum_df.pct_change().fillna(0) + 1
    return rr.dot(weights) - 1

combined_returns = calc_combined_returns(combined_df)
combined_returns.name = 'Combined'

# === 3. Load Benchmark ($SPXTR) ===
benchmark_df = norgatedata.price_timeseries(
    benchmark_symbol,
    start_date=start_date,
    end_date=end_date,
    format='pandas-dataframe',
    stock_price_adjustment_setting=norgatedata.StockPriceAdjustmentType.NONE,
    padding_setting=norgatedata.PaddingType.NONE,
    timezone='UTC'
)
benchmark_returns = benchmark_df['Close'].pct_change().dropna()
benchmark_returns = benchmark_returns.loc[start_date:end_date]
benchmark_returns.name = 'Benchmark ($SPXTR)'

# === 4. Merge All ===
df = pd.concat([
    mean_reversion.rename('Mean Reversion'),
    trend_following.rename('Trend Following'),
    combined_returns,
    benchmark_returns
], axis=1).dropna()

# === 5. Metrics Function ===
def compute_metrics(returns, benchmark=None):
    stats = {}
    daily_returns = returns
    annual_factor = 252
    cumulative = (1 + daily_returns).prod() - 1
    annual_return = (1 + cumulative) ** (annual_factor / len(daily_returns)) - 1
    volatility = daily_returns.std() * np.sqrt(annual_factor)
    sharpe = (daily_returns.mean() - risk_free_rate) / daily_returns.std() * np.sqrt(annual_factor)
    cumulative_returns = (1 + daily_returns).cumprod()
    rolling_max = cumulative_returns.cummax()
    drawdown = (cumulative_returns - rolling_max) / rolling_max
    max_dd = drawdown.min()
    calmar = annual_return / abs(max_dd) if max_dd != 0 else np.nan
    log_returns = np.log(cumulative_returns)
    x = np.arange(len(log_returns))
    slope, intercept, r_value, *_ = linregress(x, log_returns)
    stability = r_value ** 2
    threshold = 0.0
    excess = daily_returns - threshold
    downside = excess[excess < 0]
    downside_dev = np.sqrt(np.mean(downside ** 2))
    sortino = excess.mean() / downside_dev * np.sqrt(annual_factor) if downside_dev != 0 else np.nan
    omega = daily_returns[daily_returns > threshold].sum() / abs(daily_returns[daily_returns < threshold].sum())
    tail_ratio = np.percentile(daily_returns, 95) / abs(np.percentile(daily_returns, 5))
    var = np.percentile(daily_returns, 5)
    stats['Annual Return (%)'] = annual_return * 100
    stats['Cumulative Return (%)'] = cumulative * 100
    stats['Annual Volatility (%)'] = volatility * 100
    stats['Sharpe Ratio'] = sharpe
    stats['Calmar Ratio'] = calmar
    stats['Stability (R²)'] = stability
    stats['Max Drawdown (%)'] = max_dd * 100
    stats['Omega Ratio (0%)'] = omega
    stats['Sortino Ratio (0%)'] = sortino
    stats['Skew'] = skew(daily_returns)
    stats['Kurtosis'] = kurtosis(daily_returns)
    stats['Tail Ratio'] = tail_ratio
    stats['Daily VaR (95%)'] = var
    if benchmark is not None:
        aligned = pd.concat([daily_returns, benchmark], axis=1).dropna()
        slope, intercept, r, _, _ = linregress(aligned.iloc[:, 1], aligned.iloc[:, 0])
        stats['Beta'] = slope
        stats['Alpha (Ann.)'] = intercept * annual_factor
    else:
        stats['Beta'] = np.nan
        stats['Alpha (Ann.)'] = np.nan
    return stats

# === 6. Compute Metrics for All ===
mr_metrics = compute_metrics(df['Mean Reversion'], df['Benchmark ($SPXTR)'])
tf_metrics = compute_metrics(df['Trend Following'], df['Benchmark ($SPXTR)'])
combined_metrics = compute_metrics(df['Combined'], df['Benchmark ($SPXTR)'])
benchmark_metrics = compute_metrics(df['Benchmark ($SPXTR)'])

# === 7. Display Table ===
results = pd.DataFrame({
    'Mean Reversion': mr_metrics,
    'Trend Following': tf_metrics,
    'Combined': combined_metrics,
    'Benchmark ($SPXTR)': benchmark_metrics
})

preferred_order = [
    'Annual Return (%)', 'Cumulative Return (%)', 'Annual Volatility (%)', 'Sharpe Ratio', 'Calmar Ratio',
    'Stability (R²)', 'Max Drawdown (%)', 'Omega Ratio (0%)', 'Sortino Ratio (0%)',
    'Skew', 'Kurtosis', 'Tail Ratio', 'Daily VaR (95%)', 'Beta', 'Alpha (Ann.)'
]
results = results.loc[preferred_order]

# Pretty Print
styled = results.round(2).style.set_caption("Strategy Performance Metrics including Combined Model (2001-12-31 to 2025-08-12)") \
    .format(precision=2) \
    .set_table_styles([{'selector': 'caption', 'props': [('font-size', '16px'), ('font-weight', 'bold')]}]) \
    .background_gradient(axis=1, cmap='Greys', subset=results.columns)

display(styled)


_Table 6 - Strategy Performance Metrics including Combined Model (2001-12-31 to 2025-08-12)_

There are no major drawbacks in this configuration. The only trade-off is a modest reduction in total return compared to the mean reversion model alone. However, this is compensated by significant gains in consistency, drawdown control, and volatility reduction. This outcome highlights the value of combining strategies that respond to different market conditions.

Figure 11 presents the cumulative return profiles of each model and the combined result. The equity curve confirms that pairing strategies—even those with moderate correlations—can enhance the stability of returns.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import norgatedata

# === Load strategy return files ===
mean_reversion_file = r'C:\Users\Juanan\Downloads\CQF\2.Mean_Reversion\1.Zipline_Trades\1.Full_Backtest\returns_2001-12-28_to_2025-08-12_MR.csv'
trend_following_file = r'C:\Users\Juanan\Downloads\CQF\1.Futures_Trend_Following\1.Zipline_Trades\1.Full_Backtest\returns_2001-12-28_to_2025-08-12_TF.csv'

mean_reversion = pd.read_csv(mean_reversion_file, index_col=0, parse_dates=True)
trend_following = pd.read_csv(trend_following_file, index_col=0, parse_dates=True)

# Rename columns for clarity
mean_reversion.columns = ['mean_reversion']
trend_following.columns = ['trend_following']

# Combine into single DataFrame and align dates
df = pd.concat([trend_following, mean_reversion], axis=1).dropna()

# === Monthly Rebalancer ===
class MonthRebalance:
    def __init__(self, months):
        self.months = months
        self.rebalance_count = 0
        self.last_rebalance_month = None

    def rebalance(self, row, weights, date):
        current_month = date.month
        if self.last_rebalance_month != current_month:
            total = row.sum()
            rebalanced = np.multiply(weights, total)
            self.rebalance_count += 1
            self.last_rebalance_month = current_month
            return rebalanced
        else:
            return row

# === Rebalanced Portfolio Calculation ===
def calc_rebalanced_returns(returns, rebalancer, weights):
    returns = returns.copy() + 1
    cumulative = np.zeros(returns.shape)
    cumulative[0] = np.array(weights)
    rets = returns.values

    for i in range(1, len(cumulative)):
        np.multiply(cumulative[i - 1], rets[i], out=cumulative[i])
        cumulative[i] = rebalancer.rebalance(cumulative[i], weights, returns.index[i])

    cumulative_df = pd.DataFrame(cumulative, index=returns.index, columns=returns.columns)
    print("Rebalanced {} times".format(rebalancer.rebalance_count))

    cumulative_df.ffill(inplace=True)
    rr = cumulative_df.pct_change() + 1
    rebalanced_return = rr.dot(weights) - 1
    return rebalanced_return

# === Portfolio Settings ===
weights = [0.5, 0.5]  # 50% each strategy
rebalancer = MonthRebalance(months=1)
df['Combined'] = calc_rebalanced_returns(df, rebalancer, weights)

# === Add Benchmark ===
benchmark_symbol = '$SPXTR'
benchmark_label = 'S&P 500 TR'
start_date = df.index.min().strftime('%Y-%m-%d')
end_date = df.index.max().strftime('%Y-%m-%d')

benchmark_df = norgatedata.price_timeseries(
    benchmark_symbol,
    start_date=start_date,
    end_date=end_date,
    format='pandas-dataframe',
    timezone='UTC',
    fields=['Close']
)
benchmark_df.rename(columns={'Close': benchmark_label}, inplace=True)
df[benchmark_label] = benchmark_df.pct_change()

# === Cumulative Returns ===
normalized = (df + 1).cumprod()

# === Plotting ===
font = {'family': 'DejaVu Sans', 'weight': 'normal', 'size': 16}
matplotlib.rc('font', **font)

fig = plt.figure(figsize=(15, 8))
ax = fig.add_subplot(111)
ax.set_title('Mean Reversion vs Trend Following vs Combined Portfolio')

colors = {
    'trend_following': 'blue',
    'mean_reversion': 'green',
    'Combined': 'black',
    benchmark_label: 'red'
}

linestyles = {
    'trend_following': '--',
    'mean_reversion': '-.',
    'Combined': '-',
    benchmark_label: ':'
}

for col in normalized.columns:
    ax.semilogy(normalized[col], label=col, color=colors.get(col, 'grey'),
                linestyle=linestyles.get(col, '-'), linewidth=2)

ax.legend()
ax.grid(True)
plt.tight_layout()
plt.show()

_Figure 11 - Mean Reversion vs Trend Following vs Combined Portfolio and SPXTR_

While both strategies show relatively high correlation over the full sample—0.76 daily and 0.80 monthly—this does not negate the diversification benefit. Correlation is not static.

Figure 12 shows rolling correlations over time. During stress periods, such as the 2008 financial crisis and early 2020, correlation between the strategies declined—particularly on shorter horizons (63-day window). This behavior is critical: strategies that diverge during dislocations help absorb risk when it matters most.

In [ ]:
# df already has: ['trend_following','mean_reversion'] daily returns, aligned and dropna'd

# 1) Full-sample (daily) correlation
corr_full = df['trend_following'].corr(df['mean_reversion'])
print(f"Full-sample daily correlation: {corr_full:.2f}")

# 2) Rolling correlation (e.g., 63 trading days ≈ 3 months)
rolling_corr_63 = df['trend_following'].rolling(63).corr(df['mean_reversion'])

# 3) Rolling 1-year (252 trading days)
rolling_corr_252 = df['trend_following'].rolling(252).corr(df['mean_reversion'])

# 4) Monthly correlation (resample to monthly simple returns first)
monthly = (1 + df[['trend_following','mean_reversion']]).resample('M').prod() - 1
corr_monthly_full = monthly['trend_following'].corr(monthly['mean_reversion'])
print(f"Full-sample monthly correlation: {corr_monthly_full:.2f}")

# 5) Plot rolling correlations
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12,6))
ax.plot(rolling_corr_63, label='63-day rolling corr')
ax.plot(rolling_corr_252, label='252-day rolling corr', alpha=0.8)
ax.axhline(0, linestyle='--', linewidth=1)
ax.set_title('Trend vs. Mean-Reversion: Rolling Correlation')
ax.set_ylabel('Correlation')
ax.legend()
ax.grid(True)
plt.tight_layout()
plt.show()


_Figure 12 - Trend vs. Mean-Reversion: Rolling Correlation_

Going forward, additional strategies with lower or orthogonal return structures could be considered to reduce internal correlation further. But as a first implementation, this setup is simple, interpretable, and effective.
